In [ ]:
import sys
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [ ]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 50
batch_size = 100
learning_rate = 1e-3

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

trainSet = dsets.MNIST(root='./data',
                      train=True
                      transform=transforms.ToTensor(),
                      download=True)

testSet = dsets.MNIST(root='./data',
                     train=True,
                     transform=transforms.ToTensor(),
                     download=True)

In [ ]:
subset = [1,2,3,4,5]
subsetTrain = torch.utils.data.Subset(trainSet, subset)
subsetTest = torch.utils.data.Subset(testSet, subset)
trainLoader = torch.utils.data.DataLoader(dataset = subsetTrain,
                                         batch_size = batch_size,
                                         shuffle = True)

testLoader = torch.utils.data.DataLoader(dataset = subsetTest,
                                        batch_size = batch_size,
                                        shuffle = False)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,50,3) #1 Input channel (Grayscale), 50 output channels (50 features to look for), 3x3 kernel
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(50,16,3) #50 input channels (conv1 output), 16 output
        self.fc1 = nn.Linear(400, 120) #16 input (conv2) * (50/2) = 400
        self.fc2 = nn.Linear(120,10)
        
    def forward(self, x):
        out = self.pool(self.conv1(x))
        out = self.pool(self.conv2(out))
        out = torch.flatten(out, 1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
net = Net()
use_cuda = False
criterion = nn.CrossEntropyLoss()
optimizer = torch.optimAdam(net.parameters(), lr=learning_rate)        

In [ ]:
for epoch in range(num_epochs):
    for i, data in enumerate(trainLoader):
        images, labels = data
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
             %(epoch+1, num_epochs, i+1, len(trainSet)//batch_size, loss.data.item()))